# Task 5b

In [4]:
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-large')

/Users/neleht./Desktop/nosync/recent/venv_instructor/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
import pandas as pd
csv_file = "../asr-train/cv-valid-test_task4_final.csv"

df = pd.read_csv(csv_file)

hot_words = ['be careful', 'destroy' , 'stranger']

## Test

In [5]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Encode function using the model
def encode_sentences(sentences):
    return model.encode(sentences)

# Cosine similarity
def calculate_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))[0][0]

# Check similarity with hot words
def is_similar_to_hot_words(transcription_embedding, hot_word_embeddings, threshold=0.8181):
    similarities = [calculate_similarity(transcription_embedding, hw_emb) for hw_emb in hot_word_embeddings]
    return any(sim > threshold for sim in similarities), similarities

# Test on specific sample indices
sample_indices = [0, 3, 89, 508, 674, 1093, 1101, 1243, 1501, 1933, 2405, 2453,3065, 3219, 3808 ]

# Prepare sentences
transcription_sentences_sample = [
    ["Represent the transcription: ", df.loc[i, 'text']] for i in sample_indices
]
hot_words_sentences_sample = [
    ["Represent the hot word: ", hw] for hw in hot_words
]

# Encode
hot_word_embeddings_sample = encode_sentences(hot_words_sentences_sample)
transcription_embeddings_sample = encode_sentences(transcription_sentences_sample)

# Evaluate and display
print("Sample similarity results:\n")
for i, original_index in enumerate(sample_indices):
    transcription_embedding = transcription_embeddings_sample[i]
    is_similar, similarities = is_similar_to_hot_words(transcription_embedding, hot_word_embeddings_sample)
    
    print(f"Text: {df.loc[original_index, 'text']}")
    for j, hot_word in enumerate(hot_words):
        
        print(f"  Similarity with '{hot_word}': {similarities[j]:.4f}")
    print(f"  ➤ Similar to any hot word? {'Yes' if is_similar else 'No'}\n")


Sample similarity results:

Text: be careful with your prognostications said the stranger
  Similarity with 'be careful': 0.8914
  Similarity with 'destroy': 0.7295
  Similarity with 'stranger': 0.8670
  ➤ Similar to any hot word? Yes

Text: i thought that everything i owned would be destroyed
  Similarity with 'be careful': 0.7794
  Similarity with 'destroy': 0.8471
  Similarity with 'stranger': 0.7310
  ➤ Similar to any hot word? Yes

Text: the stranger seemed satisfied with the answer
  Similarity with 'be careful': 0.7596
  Similarity with 'destroy': 0.7265
  Similarity with 'stranger': 0.8764
  ➤ Similar to any hot word? Yes

Text: i had to test your courage the stranger said
  Similarity with 'be careful': 0.7913
  Similarity with 'destroy': 0.7331
  Similarity with 'stranger': 0.8777
  ➤ Similar to any hot word? Yes

Text: i had to test your courage the stranger said
  Similarity with 'be careful': 0.7913
  Similarity with 'destroy': 0.7331
  Similarity with 'stranger': 0.8777
 

## Full dataset

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Encode function from the model's website example
def encode_sentences(sentences):
   
    embeddings = model.encode(sentences)
    return embeddings
    
# Function to compute cosine similarity between two sets of embeddings
def calculate_similarity(embedding1, embedding2):
    # Reshape to 2D (1, N) to avoid the ValueError
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)
    return cosine_similarity(embedding1, embedding2)[0][0]

# Function to check if transcription is similar to hot words
def is_similar_to_hot_words(transcription_index, hot_word_embeddings, transcription_embeddings):
    transcription_embedding = transcription_embeddings[transcription_index]
    # Check similarity with each hot word
    similarities = [calculate_similarity(transcription_embedding, hot_word_embedding) for hot_word_embedding in hot_word_embeddings]
    # If any similarity is above a threshold (e.g., 0.8), return True
    
    return any(similarity > 0.8181 for similarity in similarities)


# Prepare transcription sentences for all the entries in the dataframe
transcription_sentences = [[f"Represent the transcription: ", row['text']] for _, row in df.iterrows()]
hot_words_sentences = [[f"Represent the hot word: ", hot_word] for hot_word in hot_words]

# Encode the hot words and transcriptions for the full dataset
hot_word_embeddings = model.encode(hot_words_sentences)
transcription_embeddings = model.encode(transcription_sentences)

# Process each row and check similarity with hot words
similarity_results = []

# Iterate through all the rows in the dataset
for index in range(len(df)):
    similarity = is_similar_to_hot_words(index, hot_word_embeddings, transcription_embeddings)
    print(f"Similarity check for index {index}: {similarity}")
    similarity_results.append(similarity)

# Add the similarity results to the dataframe
df['similarity'] = similarity_results

# Save the updated DataFrame with the new similarity column to a new CSV
df.to_csv('cv-valid-dev-task5.csv', index=False)

print("Similarity check completed for the full dataset and saved to 'cv-valid-dev_task5.csv'.")


Similarity check for index 0: True
Similarity check for index 1: False
Similarity check for index 2: False
Similarity check for index 3: True
Similarity check for index 4: False
Similarity check for index 5: False
Similarity check for index 6: False
Similarity check for index 7: False
Similarity check for index 8: False
Similarity check for index 9: False
Similarity check for index 10: False
Similarity check for index 11: False
Similarity check for index 12: False
Similarity check for index 13: False
Similarity check for index 14: False
Similarity check for index 15: False
Similarity check for index 16: False
Similarity check for index 17: False
Similarity check for index 18: False
Similarity check for index 19: False
Similarity check for index 20: False
Similarity check for index 21: False
Similarity check for index 22: False
Similarity check for index 23: False
Similarity check for index 24: False
Similarity check for index 25: False
Similarity check for index 26: True
Similarity che

## Check with the detechted.txt

In [37]:
import pandas as pd

# Load the CSV with the updated similarity results
df = pd.read_csv('cv-valid-dev-updated.csv')

# Read the detected.txt file containing the filenames of detected hot words
with open('detected.txt', 'r') as f:
    detected_files = [line.strip() for line in f.readlines()]

# Filter the dataframe to only include rows that match the filenames in detected.txt
detected_rows = df[df['filename'].isin(detected_files)]  # Assuming 'filename' is the column with filenames

# Display the filtered rows
print(detected_rows)

# Optionally, you can save these rows to a new CSV for further analysis
detected_rows.to_csv('detected_rows.csv', index=False)


                            filename  \
0     cv-valid-dev/sample-000000.mp3   
3     cv-valid-dev/sample-000003.mp3   
89    cv-valid-dev/sample-000089.mp3   
508   cv-valid-dev/sample-000508.mp3   
674   cv-valid-dev/sample-000674.mp3   
1093  cv-valid-dev/sample-001093.mp3   
1101  cv-valid-dev/sample-001101.mp3   
1243  cv-valid-dev/sample-001243.mp3   
1501  cv-valid-dev/sample-001501.mp3   
1933  cv-valid-dev/sample-001933.mp3   
2405  cv-valid-dev/sample-002405.mp3   
2453  cv-valid-dev/sample-002453.mp3   
3065  cv-valid-dev/sample-003065.mp3   
3219  cv-valid-dev/sample-003219.mp3   
3808  cv-valid-dev/sample-003808.mp3   

                                                   text  up_votes  down_votes  \
0     be careful with your prognostications said the...         1           0   
3     i thought that everything i owned would be des...         3           0   
89        the stranger seemed satisfied with the answer         3           0   
508        i had to test your coura

In [38]:
import pandas as pd

# Load the CSV with the updated similarity results
df = pd.read_csv('cv-valid-dev-updated.csv')

# Count the number of True and False values in the 'similarity' column
similarity_counts = df['similarity'].value_counts()

# Display the result
print(f"Count of 'True' values: {similarity_counts.get(True, 0)}")
print(f"Count of 'False' values: {similarity_counts.get(False, 0)}")


Count of 'True' values: 114
Count of 'False' values: 3962
